# Sort + Faster RCNN

### Importing Libraries

In [ ]:
from ultralytics import YOLO
import cv2
import cvzone
import math
from sort import *
import os

C:\Users\aakas\anaconda3\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
C:\Users\aakas\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


### Python Code for Real-time Car Detection and Counting with SORT Tracker


In [ ]:
import cv2
import torch
import torchvision
from torchvision.transforms import functional as F
from PIL import Image
import numpy as np
from sort import Sort

# Load the COCO instance segmentation model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Define the COCO instance category names
COCO_INSTANCE_CATEGORY_NAMES = [
    '_background_', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

# Initialize SORT tracker
tracker = Sort()

# Define the function for object detection
def detect_objects(model, frame):
    frame_pil = Image.fromarray(frame)
    frame_tensor = F.to_tensor(frame_pil)
    frame_tensor = frame_tensor.unsqueeze(0)
    with torch.no_grad():
        prediction = model(frame_tensor)
    boxes = prediction[0]['boxes'].detach().cpu().numpy()
    labels = prediction[0]['labels'].detach().cpu().numpy()
    scores = prediction[0]['scores'].detach().cpu().numpy()

    # Initialize variables for counting cars
    car_counter = 0
    car_boxes = []

    for box, label, score in zip(boxes, labels, scores):
        if score > 0.5 and COCO_INSTANCE_CATEGORY_NAMES[label] == 'car':
            box = box.astype(int)
            car_counter += 1
            car_boxes.append(box)
            # Visualize bounding boxes
            cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)

    return car_counter, car_boxes

# Load the video file
video_path = 'C:/Users/aakas/Desktop/VIDEO-2024-03-16-23-08-40.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# Initialize variables for car counting
car_count = 0

# Loop through the video frames and perform object detection
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection
    car_counter, car_boxes = detect_objects(model, frame)

    # Update SORT tracker with new car detections
    tracked_cars = tracker.update(np.array(car_boxes))

    # Visualize tracked cars
    for car in tracked_cars:
        x1, y1, x2, y2, track_id = car.astype(int)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, str(track_id), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # Update car count
    car_count = len(tracked_cars)

    # Display car count on frame
    cv2.putText(frame, f"Car Count: {car_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the frame
    cv2.imshow('Frame', frame)

    # Exit if 'q' is pressed
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release video capture object
cap.release()

# Close OpenCV windows
cv2.destroyAllWindows()

print(f"Total car count: {car_count}")


C:\Users\aakas\anaconda3\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
C:\Users\aakas\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
C:\Users\aakas\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\aakas\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DE

Total car count: 5


Above count was no. of cars in particular frame